In [67]:
using Plots
gr()

Plots.GRBackend()

In [68]:
using NPZ
using CSV
using DataFrames
using DelimitedFiles

In [69]:
vars = npzread("./augmented_train_test_data.npz")

Dict{String, Array{Float64, 3}} with 3 entries:
  "aug_y_calib" => [0.012299 0.0115164 … 0.0985421 0.436506; 0.012299 0.0115164…
  "aug_y_train" => [0.012299 0.0115164 … 0.0985421 0.436506; 0.012299 0.0115164…
  "aug_y_test"  => [0.012299 0.0115164 … 0.0985421 0.436506; 0.012299 0.0115164…

In [70]:
keys(vars)

KeySet for a Dict{String, Array{Float64, 3}} with 3 entries. Keys:
  "aug_y_calib"
  "aug_y_train"
  "aug_y_test"

In [71]:
size(vars["aug_y_calib"])

(90, 49, 39)

In [72]:
aug_y_train = vars["aug_y_train"]
aug_y_calib = vars["aug_y_calib"]
aug_y_test = vars["aug_y_test"]
size(aug_y_train)

(90, 49, 183)

In [73]:
size(aug_y_train)[3]

183

Gather train data into a big matrix (stacked horizontally) based on time and theta.

In [74]:
tVars = npzread("./tMinMaxTrain.npz")

Dict{String, Vector{Int64}} with 2 entries:
  "tMinTrainIdx" => [11, 31, 34, 19, 16, 15, 27, 39, 42, 20  …  25, 24, 16, 19,…
  "tMaxTrainIdx" => [79, 89, 89, 89, 89, 89, 89, 89, 89, 89  …  89, 89, 89, 89,…

In [75]:
# these indices are off by one since Julia uses 1-indexing

In [76]:
tMinTrainIdx = tVars["tMinTrainIdx"]
tMaxTrainIdx = tVars["tMaxTrainIdx"];

In [77]:
tVarsTest = npzread("./tMinMaxTest.npz")
tMinTestIdx = tVarsTest["tMinTestIdx"]
tMaxTestIdx = tVarsTest["tMaxTestIdx"];

In [181]:
tMinTestIdx, tMaxTestIdx

([20, 30, 13, 24, 42, 30, 15, 14, 24, 25  …  16, 13, 24, 19, 16, 36, 25, 22, 26, 15], [89, 89, 89, 89, 89, 89, 89, 89, 89, 89  …  89, 69, 89, 89, 89, 89, 89, 89, 89, 89])

In [78]:
train_lengths = (tVars["tMaxTrainIdx"] - tVars["tMinTrainIdx"]) .+ 1
total_train_lengths = sum(train_lengths)

12119

In [79]:
cumulative_lengths = [0; cumsum(train_lengths)]
cumulative_lengths'

1×184 adjoint(::Vector{Int64}) with eltype Int64:
 0  69  128  184  255  329  404  467  …  11841  11910  11984  12041  12119

In [80]:
90*183

16470

In [81]:
aug_y_train[:, 1:40, 1]

90×40 Matrix{Float64}:
 0.012299   0.0115164  0.0107423  …  0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423  …  0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115164  0.0107423     0.00126086  0.00167312  0.00212049
 0.012299   0.0115533  0.0109098     0.00126086  0.00167312  0.00212049
 0.0139361  0.0119144  0.0109098     0.00164631  0.00198896  0.00212049
 0.0147729  0.0120175  0.0109098  …  0.0022923   0.00267987  0.00252075
 0.0284808  0.0252072  0.0232569     0.0022923   0.00267987  0.00330972
 0.0291709  0.0299034  0.0308699     0.0022923   0.00267987  0.00330972
 ⋮                                ⋱      

In [82]:
truncatedData = zeros(40, total_train_lengths)
for i in 1:size(aug_y_train)[3]
#     println(cumulative_lengths[i])
    truncatedData[1:40, (cumulative_lengths[i] + 1):(cumulative_lengths[i+1])] = Matrix(aug_y_train[(tMinTrainIdx[i] + 1):(tMaxTrainIdx[i] + 1), 1:40, i]')
#     truncatedData[41:46, (cumulative_lengths[i] + 1):(cumulative_lengths[i+1])] = Matrix(aug_y_train[(tMinTrainIdx[i]+1):(tMaxTrainIdx[i]+1), [41, 45, 46, 47, 48, 49], i]')
end
truncatedData

40×12119 Matrix{Float64}:
 0.0284808   0.0291709   0.0388966   …  0.572686  0.576868  0.585561
 0.0252072   0.0299034   0.0391994      0.581194  0.584569  0.595226
 0.0232569   0.0308699   0.0399628      0.589068  0.592027  0.603845
 0.0224707   0.0320601   0.0411533      0.59659   0.599448  0.611774
 0.0226899   0.0334639   0.0427376      0.604043  0.607042  0.619371
 0.0237557   0.0350711   0.0446823   …  0.611709  0.615017  0.626993
 0.0255091   0.0368713   0.0469542      0.619871  0.62358   0.634998
 0.027782    0.0388897   0.0495509      0.628821  0.632906  0.643794
 0.0304186   0.0410797   0.0524889      0.638926  0.643309  0.653858
 0.0331787   0.0434209   0.0556211      0.650571  0.655102  0.665573
 0.0359948   0.0458409   0.0588175   …  0.664018  0.668518  0.679201
 0.0387234   0.0483401   0.0619411      0.679412  0.683702  0.69487
 0.0412283   0.0508413   0.0648388      0.696658  0.700649  0.712468
 ⋮                                   ⋱                      
 0.0298607   0.04

In [131]:
cumulative_lens_2 = [0; cumsum(90 * ones(Int, 183))]
cumulative_lens_2'

1×184 adjoint(::Vector{Int64}) with eltype Int64:
 0  90  180  270  360  450  540  630  …  16110  16200  16290  16380  16470

In [132]:
truncatedData2 = zeros(40, 90 * 183)
for i in 1:size(aug_y_train)[3]
    truncatedData2[1:40, (cumulative_lens_2[i] + 1):(cumulative_lens_2[i+1])] = Matrix(aug_y_train[:, 1:40, i]')
end

In [133]:
truncatedData2

40×16470 Matrix{Float64}:
  0.012299      0.012299      0.012299     …  0.572686  0.576868  0.585561
  0.0115164     0.0115164     0.0115164       0.581194  0.584569  0.595226
  0.0107423     0.0107423     0.0107423       0.589068  0.592027  0.603845
  0.00997846    0.00997846    0.00997846      0.59659   0.599448  0.611774
  0.00922691    0.00922691    0.00922691      0.604043  0.607042  0.619371
  0.0084896     0.0084896     0.0084896    …  0.611709  0.615017  0.626993
  0.00776845    0.00776845    0.00776845      0.619871  0.62358   0.634998
  0.00706538    0.00706538    0.00706538      0.628821  0.632906  0.643794
  0.00638227    0.00638227    0.00638227      0.638926  0.643309  0.653858
  0.00572098    0.00572098    0.00572098      0.650571  0.655102  0.665573
  0.00508332    0.00508332    0.00508332   …  0.664018  0.668518  0.679201
  0.00447104    0.00447104    0.00447104      0.679412  0.683702  0.69487
  0.00388585    0.00388585    0.00388585      0.696658  0.700649  0.712468


In [83]:
# truncatedData[:, 11911:end]

In [85]:
XTrainScaled = aug_y_train[:, [41, 45, 46, 47, 48, 49], :] # 90 x 6 x 183

90×6×183 Array{Float64, 3}:
[:, :, 1] =
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 ⋮                                           ⋮
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.436506
 0.5675  1.0  0.272727  0.120833  0.0985421  0.43

In [163]:
XTestScaled = aug_y_test[:, [41, 45, 46, 47, 48, 49], :]

90×6×40 Array{Float64, 3}:
[:, :, 1] =
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 ⋮                                           ⋮
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436506
 0.7825  1.0  0.272727  0.120833  0.0985421  0.436

In [106]:
X_train_orig = 2 * (Matrix(XTrainScaled[1, :, :]') .- 0.5 * ([1, 1, 1, 1, 1, 1]'))
X_train_orig

183×6 Matrix{Float64}:
  0.135   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.095   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.755  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.485   1.0  -0.454545  -0.758333  -0.802916  -0.126989
  0.745   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.455   1.0  -0.454545  -0.758333  -0.802916  -0.126989
  0.885  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.875   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.475   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.675   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.225   1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.585  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.805  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
  ⋮                                              ⋮
 -0.35    1.0   0.454545  -0.758333  -0.802916  -0.126989
  0.65    1.0  -0.272727  -0.294815  -0.988714   0.874387
  0.15   -1.0  -0.272727  -0.294815  -0.988714   0.87438

In [164]:
X_test_orig = 2 * (Matrix(XTestScaled[1, :, :]') .- 0.5 * ([1, 1, 1, 1, 1, 1]'))

40×6 Matrix{Float64}:
  0.565   1.0  -0.454545  -0.758333  -0.802916  -0.126989
  0.635  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
  0.835   1.0  -0.454545  -0.758333  -0.802916  -0.126989
  0.165  -1.0  -0.454545  -0.758333  -0.802916  -0.126989
 -0.655  -1.0   0.636364  -0.294815  -0.988714   0.874387
  0.385  -1.0   0.636364  -0.294815  -0.988714   0.874387
 -0.205   1.0   0.636364  -0.294815  -0.988714   0.874387
  0.945   1.0   0.636364  -0.294815  -0.988714   0.874387
 -0.695  -1.0   0.636364  -0.294815  -0.988714   0.874387
 -0.975  -1.0  -0.454545  -0.294815  -0.988714   0.874387
  0.805   1.0  -0.454545  -0.294815  -0.988714   0.874387
  0.345   1.0  -0.454545  -0.294815  -0.988714   0.874387
  0.975   1.0  -0.454545  -0.294815  -0.988714   0.874387
  ⋮                                              ⋮
  0.95    1.0   0.636364  -0.294815  -0.988714   0.874387
 -0.75   -1.0   0.636364  -0.294815  -0.988714   0.874387
 -0.35    1.0   0.636364  -0.294815  -0.988714   0.874387

In [120]:
# repeat(X_train_orig, inner=(90, 1))

16470×6 Matrix{Float64}:
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 ⋮                                             ⋮
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294

In [112]:
X_train_orig_repeat = Matrix(repeat(X_train_orig, inner=(90, 1))')

6×16470 Matrix{Float64}:
  0.135      0.135      0.135     …   0.25       0.25       0.25
  1.0        1.0        1.0           1.0        1.0        1.0
 -0.454545  -0.454545  -0.454545     -0.272727  -0.272727  -0.272727
 -0.758333  -0.758333  -0.758333     -0.294815  -0.294815  -0.294815
 -0.802916  -0.802916  -0.802916     -0.988714  -0.988714  -0.988714
 -0.126989  -0.126989  -0.126989  …   0.874387   0.874387   0.874387

In [134]:
X_train_rescaled_repeat = reshape(X_train_orig_repeat, 6, 90, 183);

In [135]:
size(X_train_rescaled_repeat)

(6, 90, 183)

In [136]:
# ξTrainScaled = zeros(total_train_lengths, 6)
ξTrainScaled = zeros(90*183, 6)
for i in 1:size(aug_y_train)[3]
#     ξTrainScaled[(cumulative_lengths[i] + 1):(cumulative_lengths[i+1]), :] = [(tMinTrainIdx[i] + 1):(tMaxTrainIdx[i] + 1), :,  i]
    ξTrainScaled[(cumulative_lens_2[i] + 1):(cumulative_lens_2[i+1]), :] = Matrix(X_train_rescaled_repeat[:, :,  i]')
end

In [137]:
ξTrainScaled

16470×6 Matrix{Float64}:
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 0.135  1.0  -0.454545  -0.758333  -0.802916  -0.126989
 ⋮                                             ⋮
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294815  -0.988714   0.874387
 0.25   1.0  -0.272727  -0.294

In [138]:
include("./kleUtils.jl")

subsolve (generic function with 1 method)

In [139]:
YCenteredTrain, YMeanTrain = getCenteredSolution(truncatedData2)
wts, wtRoot, wtInvRoot = getWeights(collect(1:40))

([0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5], [0.7071067811865476, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7071067811865476], [1.414213562373095, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.414213562373095])

In [140]:
Q, λ, kt = getSortedEigenpairsFromSVD(YCenteredTrain,
                                      collect(1:40);
                                      useFullGrid=1,
                                      getAllModes=0,
                                      weightFunction=getWeights
                                      )

([-0.14130244643578974; -0.14252945273163148; … ; -0.13928847572447325; -0.13552075201138308;;], [2.5324986821669695], 1)

In [141]:
λ

1-element Vector{Float64}:
 2.5324986821669695

In [142]:
pceOrder=3

3

In [143]:
ζTrain = getUncorrelatedRV(wts, YCenteredTrain, Q, λ)

16470×1 Matrix{Float64}:
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.011453247037718
  1.0058376063442693
  0.9726250839752535
  0.9511221903243536
  0.8997436406651146
  0.8562369634252818
  ⋮
 -1.66559766057008
 -1.7215321284429135
 -1.7418854749199244
 -1.8108041384442497
 -1.812101013739119
 -1.8679671663180781
 -1.9094979058695476
 -1.9445383811297365
 -1.9824180929992574
 -2.0229317727442835
 -2.0373557519129992
 -2.0945031222691552

In [144]:
ATrain = PrepCaseA(X_train_orig; order=pceOrder, dims=6, family="Legendre")

183×84 Matrix{Float64}:
 1.0  -0.219951  -1.39069  -1.31347   …   0.832097   -1.83061    -0.519491
 1.0  -0.219951  -1.39069  -1.31347       0.856391   -1.88406     0.371349
 1.0  -0.219951  -1.39069  -1.31347      -0.625025   -1.37505     0.149691
 1.0  -0.219951  -1.39069  -1.31347       0.259072   -0.569958    1.17019
 1.0  -0.219951  -1.39069  -1.31347      -0.585415    1.28791    -0.221624
 1.0  -0.219951  -1.39069  -1.31347   …   0.333539   -0.733785    1.18267
 1.0  -0.219951  -1.39069  -1.31347      -1.18802    -2.61363     1.07255
 1.0  -0.219951  -1.39069  -1.31347      -1.14154     2.51139    -0.958568
 1.0  -0.219951  -1.39069  -1.31347       0.284422   -0.625729    1.17622
 1.0  -0.219951  -1.39069  -1.31347      -0.322932    0.71045     0.644592
 1.0  -0.219951  -1.39069  -1.31347   …   0.74654    -1.64239     0.817599
 1.0  -0.219951  -1.39069  -1.31347      -0.02348    -0.0516559   0.997437
 1.0  -0.219951  -1.39069  -1.31347      -0.830997   -1.82819    -0.255713
 ⋮   

In [145]:
nt = size(ATrain, 2)

84

In [146]:
kt = size(ζTrain, 2)

1

In [155]:
ATrain \ ζTrain[50:90:end, 1]

84-element Vector{Float64}:
 -0.0005084470509020402
 -0.0006027533182908164
  0.0008396791831387715
  0.00033706188782674077
 -0.00523816241013143
  0.008978252984394551
 -0.013746631217372214
 -0.0004932839510555566
  0.0010390436089181593
 -0.0009909603764706259
  0.0002907553481214601
 -0.0005364521025892805
  0.00026232177153474244
  ⋮
 -0.11146629188659747
  0.029754462687547306
 -0.009299213121620851
  0.010574580068672193
  0.028374771402874383
 -0.030738401863664937
 -0.06755710844320463
 -0.001612802781820227
 -0.04324578802924325
  0.04065395335581726
  0.03454824305836729
 -0.02502208263500953

In [156]:
bβ_all = zeros(kt, nt, 90);
for timeIdx in 1:90
    for k in 1:kt
        bβ_all[k, :, timeIdx] = ATrain \ ζTrain[timeIdx:90:end, k]
    end
end

In [158]:
# Making predictions. Feed in correct position of tMinTrainIdx and predict only upto tMaxTrainIdx

In [159]:
test_lengths = (tMaxTestIdx - tMinTestIdx) .+ 1
total_test_lengths = sum(test_lengths)

2671

In [160]:
size(aug_y_test)

(90, 49, 40)

In [168]:
klModes = Q .* sqrt.(λ)'

40×1 Matrix{Float64}:
 -0.2248662591496893
 -0.22681889565853014
 -0.229328831546678
 -0.23216275570702602
 -0.23502166388486642
 -0.23791731584481246
 -0.24120022766815616
 -0.24452228911853344
 -0.24774997867603496
 -0.2509964535336098
 -0.2541123673264641
 -0.2569590995731971
 -0.25981849708902355
  ⋮
 -0.2660788099894494
 -0.26350462854328294
 -0.26054840177205074
 -0.2571161038230398
 -0.2532578086667528
 -0.2488971774985493
 -0.2441046213999607
 -0.23895165961857082
 -0.2334264479064565
 -0.22761903239245815
 -0.221661261137878
 -0.21566537105782083

In [173]:
klModes

40×1 Matrix{Float64}:
 -0.2248662591496893
 -0.22681889565853014
 -0.229328831546678
 -0.23216275570702602
 -0.23502166388486642
 -0.23791731584481246
 -0.24120022766815616
 -0.24452228911853344
 -0.24774997867603496
 -0.2509964535336098
 -0.2541123673264641
 -0.2569590995731971
 -0.25981849708902355
  ⋮
 -0.2660788099894494
 -0.26350462854328294
 -0.26054840177205074
 -0.2571161038230398
 -0.2532578086667528
 -0.2488971774985493
 -0.2441046213999607
 -0.23895165961857082
 -0.2334264479064565
 -0.22761903239245815
 -0.221661261137878
 -0.21566537105782083

In [191]:
klModes = Q .* sqrt.(λ)'
YTestPreds = Vector{Matrix{Float64}}(undef, 40)
for i in 1:size(aug_y_test, 3)
    ΨTest = transpose(PrepCaseA(X_test_orig[i, :]'; order=pceOrder, dims=6))
    YTest_sim = zeros(40, tMaxTestIdx[i] - tMinTestIdx[i] + 1)
    for j in 1:(tMaxTestIdx[i] - tMinTestIdx[i] + 1)
        YTest_sim[:, j] = klModes * bβ_all[:, :, tMinTestIdx[i] + 1 + (j - 1)] * ΨTest + YMeanTrain
    end
    YTestPreds[i] =  YTest_sim
end

In [192]:
size(YTestPreds[1])

(40, 70)

In [193]:
size(YTestPreds[3])

(40, 77)

In [195]:
typeof(YTestPreds)

Vector{Matrix{Float64}} (alias for Array{Array{Float64, 2}, 1})

In [199]:
"$(1)"

"1"

In [202]:
npzwrite("./kle_predictions_test_sims.npz", Dict(["$(i)" => YTestPreds[i] for i in 1:40]))